In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\Data1.csv')
df.head()

,label,description
0,Amazon S3,"['public', 'access', 'set', 'bucket', 'policy'..."
1,Amazon S3,"['figure', 'display', 'website', 'domain', 'Er..."
2,Amazon S3,"['test', 'ignore']"
3,Amazon S3,"['account', 'suspended', 'without', 'Notificat..."
4,Amazon S3,"['Unable', 'read', '1000', 'objects', 'bucket'..."


In [2]:
df['label'].value_counts()

Amazon S3                   53174
Amazon EC2                  46535
Amazon RDS                  29525
Amazon CloudFront           22314
Amazon Elastic Beanstalk    22246
Amazon VPC                  21809
Amazon Lambda                8439
Amazon SNS                   4530
Name: label, dtype: int64

In [3]:
tf = pd.read_csv('C:\\Users\\Aruna\\Documents\\ACMS-IID\\Code\\Data Cleaning\\TestData.csv')
tf.head()

,label,description
0,Amazon EC2,"['spot', 'request', 'terminated', 'says', 'ini..."
1,Amazon EC2,"['bump', 'needs', 'looked']"
2,Amazon EC2,['bump']
3,Amazon EC2,"['insufficient', 'data', 'instance', 'status',..."
4,Amazon EC2,"['lamba', 'kms', 'issues', 'encountering', 'de..."


In [4]:
tf['label'].value_counts()

Amazon S3                   100
Amazon SNS                   97
Amazon Lambda                87
Amazon RDS                   71
Amazon Elastic Beanstalk     55
Amazon EC2                   49
Amazon CloudFront            38
Amazon VPC                   34
Name: label, dtype: int64

In [5]:
# from sklearn.model_selection import train_test_split

X_train = df['description']  # this time we want to look at the text
y_train = df['label']

X_test = tf['description']
y_test = tf['label']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(208572, 132702)

In [7]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [8]:
from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('rfc',LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [9]:
predictions = text_clf.predict(X_test)

In [10]:
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[27  2  0  2  0  6  0  1]
 [ 1 44  0  0  1  3  0  0]
 [ 0 10 38  1  2  2  0  2]
 [ 8 13  4 56  0  5  0  1]
 [ 0  9  1  1 54  2  1  3]
 [ 5  9  0  0  4 81  1  0]
 [ 0  6  1  2  3  4 78  3]
 [ 0  5  1  0  4  0  0 24]]


In [11]:
print(metrics.classification_report(y_test,predictions))

                          precision    recall  f1-score   support

       Amazon CloudFront       0.66      0.71      0.68        38
              Amazon EC2       0.45      0.90      0.60        49
Amazon Elastic Beanstalk       0.84      0.69      0.76        55
           Amazon Lambda       0.90      0.64      0.75        87
              Amazon RDS       0.79      0.76      0.78        71
               Amazon S3       0.79      0.81      0.80       100
              Amazon SNS       0.97      0.80      0.88        97
              Amazon VPC       0.71      0.71      0.71        34

               micro avg       0.76      0.76      0.76       531
               macro avg       0.76      0.75      0.74       531
            weighted avg       0.80      0.76      0.77       531



In [12]:
print(metrics.accuracy_score(y_test,predictions))

0.7570621468926554
